## Data transformations

### Reformat Data 

#### codecs

Encoders and decoders for converting text between different representations.

-https://pymotw.com/3/codecs/index.html

In [41]:
import binascii


def to_hex(t, nbytes):
    """Format text t as a sequence of nbyte long values
    separated by spaces.
    """
    chars_per_item = nbytes * 2
    hex_version = binascii.hexlify(t)
    return b' '.join(
        hex_version[start:start + chars_per_item]
        for start in range(0, len(hex_version), chars_per_item)
    )


if __name__ == '__main__':
    print(to_hex(b'abcdef', 1))
    print(to_hex(b'abcdef', 2))

b'61 62 63 64 65 66'
b'6162 6364 6566'


In [42]:
text = 'français'
encoded = text.encode('utf-8')
decoded = encoded.decode('utf-8')

print('Original :', repr(text))
print('Encoded  :', to_hex(encoded, 1), type(encoded))
print('Decoded  :', repr(decoded), type(decoded))

Original : 'français'
Encoded  : b'66 72 61 6e c3 a7 61 69 73' <class 'bytes'>
Decoded  : 'français' <class 'str'>


#### Base64
Base64 encoding is a type of conversion of bytes into ASCII characters.  data sometimes need to be sent as text so it won't be corrupted. 
- [Encoding and Decoding Base64 Strings in Python](https://stackabuse.com/encoding-and-decoding-base64-strings-in-python/) 

In [1]:
import base64

In [2]:
message = "Python is fun"
message_bytes = message.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')

print(base64_message)

UHl0aG9uIGlzIGZ1bg==


In [3]:
def b64_decode(payload):
    # base64_bytes = base64_message.encode('utf-8')
    base64_bytes = payload
    message_bytes = base64.b64decode(base64_bytes)
    message = message_bytes.decode('utf-8')

    # print(message_bytes, '\n', message)
    return message

print(b64_decode(base64_message))

Python is fun


#### Unicode Data and Network Communication¶

```
$ python codecs_socket.py "wen 光"

Sending : 'wen 光'
Writing : b'wen \xe5\x85\x89'
Reading : b'wen \xe5\x85\x89'
Reading : b''
Received: 'wen 光'


```


### Hash/Digest Data 

- [Cryptographic Hashing](https://pymotw.com/3/hashlib/index.html)
- [Hashlib in Python](https://docs.python.org/3/library/hashlib.html)

Since hashlib is “backed” by OpenSSL, all of the algorithms provided by that library are available, including:

- md5
- sha1
- sha224
- sha256
- sha384
- sha512

In [44]:
# md5
import hashlib
sent = b"Covid-19 is such a deadly epidemic"
h = hashlib.md5()
h.update(sent)
print(h.hexdigest())

d80d7022d3af31e772fae4a0abc9b619


In [45]:
# sha256
import hashlib
sent = b"Covid-19 is such a deadly epidemic"
m = hashlib.sha256()
m.update(sent)
# m.update(b" 2020 is a challenging year for humanity")
digest = m.digest()

In [22]:
digest

b'\xcc\xe6Yd\x8f\x10\xcc\x95\xc2\xfd\xbd\x9d\x04\xc3*6b\xc9/\xa5[\xc0\xbc\xd10`\xb94\xd4\xfc\x1bH'

In [28]:
m.name, m.digest_size, m.block_size

('sha256', 32, 64)

In [25]:
hashlib.sha256(sent).hexdigest()

'cce659648f10cc95c2fdbd9d04c32a3662c92fa55bc0bcd13060b934d4fc1b48'

In [24]:
hashlib.sha224(sent).hexdigest()

'5ec5ec3077f4930594ffd96c565cf84aa8bcb84b7b0aafa8ccd13be0'

In [26]:
h = hashlib.new('ripemd160')
h.update(sent)
h.hexdigest()

'173832593a72c04dca97db5b24921d5543c1f6c8'

In [27]:
h.name

'ripemd160'

In [29]:
from hashlib import blake2b
h = blake2b()
h.update(sent)
h.hexdigest()

'63b42daf11d96f6400d02a208217bcd16034f8bc9a3976419dab1dd0e837081cfdaa9203669c884ef238e8edf6473e2382eb1b98e638f1cf97b2b5b9c764967e'

As a practical example, a web application can symmetrically sign cookies sent to users and later verify them to make sure they weren’t tampered with:

In [31]:
from hashlib import blake2b
from hmac import compare_digest

SECRET_KEY = b'pseudorandomly generated server secret key'
AUTH_SIZE = 16

def sign(cookie):
    h = blake2b(digest_size=AUTH_SIZE, key=SECRET_KEY)
    h.update(cookie)
    return h.hexdigest().encode('utf-8')

def verify(cookie, sig):
    good_sig = sign(cookie)
    return compare_digest(good_sig, sig)

cookie = b'user-alice'
sig = sign(cookie)
print("{0},{1}".format(cookie.decode('utf-8'), sig))
# user-alice,b'43b3c982cf697e0c5ab22172d1ca7421'
print(verify(cookie, sig))

print(verify(b'user-bob', sig))

print(verify(cookie, b'0102030405060708090a0b0c0d0e0f00'))

user-alice,b'43b3c982cf697e0c5ab22172d1ca7421'
True
False
False


By setting salt parameter users can introduce randomization to the hash function. Randomized hashing is useful for protecting against collision attacks on the hash function used in digital signatures.

In [32]:
import os
from hashlib import blake2b
msg = b'Hello Python'
# Calculate the first hash with a random salt.
salt1 = os.urandom(blake2b.SALT_SIZE)
h1 = blake2b(salt=salt1)
h1.update(msg)
# Calculate the second hash with a different random salt.
salt2 = os.urandom(blake2b.SALT_SIZE)
h2 = blake2b(salt=salt2)
h2.update(msg)
# The digests are different.
h1.digest() != h2.digest()

True

In [33]:
h1.hexdigest(), h2.hexdigest()

('a65fecefac2df941f2b3a5d672fcc5a0141b5392c5d86f03d4329631a3b604d0e628f9892de14a8d4784f1e8f1fea13b09c7a2d71de74fc68fe2548961b8c186',
 '6b72c7112e5dc7b81ec478dda0da62f9453b19a155d8d05e8271f5cc1b9be6672b28e058d0f975553d286173d0aed7ee6d67462b73b07fd3eae8be515e735dd4')

### Secure Data 

- [Encryption and Decryption in Python](https://nitratine.net/blog/post/encryption-and-decryption-in-python/)

`python -m pip install cryptography`

Symmetric encryption is when a key is used to encrypt and decrypt a message, so whoever encrypted it can decrypt it. The only way to decrypt the message is to know what was used to encrypt it; kind of like a password.

In [34]:
import cryptography

In [35]:
from cryptography.fernet import Fernet
key = Fernet.generate_key()
key

b'sKnL7D125K1XL86wM5BuTgGam-yHzxsJw0DGlj38g5o='

In [37]:
with open('key.key', 'wb') as f:
    f.write(key) # The key is type bytes still

In [38]:
with open('key.key', 'rb') as f:
    key = f.read() # The key will be type bytes

In [39]:
key

b'sKnL7D125K1XL86wM5BuTgGam-yHzxsJw0DGlj38g5o='

In [41]:
key.decode('utf-8')

'sKnL7D125K1XL86wM5BuTgGam-yHzxsJw0DGlj38g5o='

In [42]:
key.decode('ascii')

'sKnL7D125K1XL86wM5BuTgGam-yHzxsJw0DGlj38g5o='

Generating a Key From A Password

In [43]:
import base64
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password_provided = "password" # This is input in the form of a string
password = password_provided.encode() # Convert to type bytes
salt = b'salt_' # CHANGE THIS - recommend using a key from os.urandom(16), must be of type bytes
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=10000,
    backend=default_backend()
)
key = base64.urlsafe_b64encode(kdf.derive(password)) # Can only use kdf once

In [44]:
key

b'BgZmHWY8OXdcNk-5QdFDxE2rS2dl_C2OX9zWHvu23mw='

In [45]:
# Encrypting
from cryptography.fernet import Fernet
message = "Hello Python".encode()

f = Fernet(key)
encrypted = f.encrypt(message)

In [46]:
encrypted

b'gAAAAABefsPptBmg8WLWvmFhmzhVU_3ZCnknGoQOvYnkYxw-7wh771YdWsH-2b3wcU2qzce5XRuV3hNfkDPXjyUENZIEyfYhCg=='

In [49]:
# decrypting
f = Fernet(key)
decrypted = f.decrypt(encrypted)
decrypted.decode('utf-8')

'Hello Python'

### Squeeze Data 

- [Data Compression and Archiving](https://pymotw.com/3/compression.html)
- [Compressing and Extracting Files in Python](https://code.tutsplus.com/tutorials/compressing-and-extracting-files-in-python--cms-26816)

In [1]:
!pwd
!ls

/home/devopsgong/projects/py4kids/lesson-65-auth
data_trans.ipynb  jwt.ipynb  jwt.ipynb.zip


In [2]:
import zipfile

fname = 'jwt.ipynb.zip'

with zipfile.ZipFile(fname, 'w') as z:
    z.write('jwt.ipynb', compress_type=zipfile.ZIP_DEFLATED)

In [3]:
!ls

data_trans.ipynb  jwt.ipynb  jwt.ipynb.zip


In [4]:
!ls -l

total 44
-rw-r--r-- 1 devopsgong devopsgong 15200 Mar 28 14:22 data_trans.ipynb
-rw-r--r-- 1 devopsgong devopsgong 18601 Mar 27 23:38 jwt.ipynb
-rw-r--r-- 1 devopsgong devopsgong  4341 Mar 28 14:24 jwt.ipynb.zip


In [21]:
# Compressing Multiple Files
import os
import zipfile

dir_path = "."
with zipfile.ZipFile('ipynb.zip', 'w') as nb_zip:
    for folder, subfolders, files in os.walk(dir_path):
        for file in files:
            if file.endswith('.ipynb') and "-checkpoint" not in file:
                nb_zip.write(
                    os.path.join(folder, file), 
                    os.path.relpath(os.path.join(folder,file), dir_path),
                    compress_type = zipfile.ZIP_DEFLATED
                )


In [30]:
!ls -l 

total 56
-rw-r--r-- 1 devopsgong devopsgong 16515 Mar 28 14:43 data_trans.ipynb
-rw-r--r-- 1 devopsgong devopsgong  8271 Mar 28 14:44 ipynb.zip
-rw-r--r-- 1 devopsgong devopsgong 18601 Mar 27 23:38 jwt.ipynb
drwxr-xr-x 2 devopsgong devopsgong  4096 Mar 28 14:44 tmp


In [25]:
# extract all files
import zipfile
with zipfile.ZipFile('ipynb.zip') as nb_zip:
    nb_zip.extractall("./tmp")

In [31]:
!ls -l tmp

total 40
-rw-r--r-- 1 devopsgong devopsgong 16515 Mar 28 14:44 data_trans.ipynb
-rw-r--r-- 1 devopsgong devopsgong 18601 Mar 28 14:44 jwt.ipynb


In [32]:
# extract a single file
import zipfile
with zipfile.ZipFile('ipynb.zip') as nb_zip:
    nb_zip.extract("jwt.ipynb", "./tmp2")

In [33]:
!ls -l tmp2

total 20
-rw-r--r-- 1 devopsgong devopsgong 18601 Mar 28 14:47 jwt.ipynb


In [39]:
# get file info
import zipfile
with zipfile.ZipFile('ipynb.zip') as nb_zip:
    for file in nb_zip.namelist():
        print(file)
        fi = nb_zip.getinfo(file)
        print(fi.date_time, fi.file_size, fi.compress_size)

jwt.ipynb
(2020, 3, 27, 23, 38, 34) 18601 4225
data_trans.ipynb
(2020, 3, 28, 14, 43, 48) 16515 3822
